In [76]:
import os
import pandas as pd
from datetime import datetime, timedelta
import pandas_datareader as dr
import yfinance as yf
import numpy as np
import glob
from collections import Counter
from ast import literal_eval


In [38]:
os.chdir('..')
os.chdir('..')
cwd = os.getcwd()
print(cwd)

/home/mfclinton/Documents/Repos/Subreddit-Stock-Prediction


In [39]:
path = r'data' # use your path
all_files = glob.glob(path + "/*.jsonl")

tickCnt2 = Counter()

for filename in all_files:
    print(filename)
    df = pd.read_json(filename,lines=True)
    for x in df['tickers']:
        if len(x) > 0:
            y, z = x[0]
            tickCnt2[y] += 1

data/stockmarket_comments.jsonl
data/options_comments.jsonl
data/investing_comments.jsonl
data/stocks_submission.jsonl
data/securityanalysis_comments.jsonl
data/stockmarket_submission.jsonl
data/options_submission.jsonl
data/daytrading_submission.jsonl
data/daytrading_comments.jsonl
data/economy_submission.jsonl
data/stocks_comments.jsonl
data/dividends_submission.jsonl
data/investing_submission.jsonl
data/dividends_comments.jsonl
data/securityanalysis_submission.jsonl
data/economy_comments.jsonl


In [40]:
top_n = 11
top_list = tickCnt2.most_common(top_n)
print(top_list)

[('TSLA', 3382), ('AMD', 2875), ('AAPL', 2140), ('AMZN', 1981), ('MSFT', 1670), ('TD', 1597), ('GOOG', 1543), ('NIO', 1299), ('GE', 1281), ('FB', 1178), ('BABA', 1048)]


In [90]:
blacklist = ["TD"]
top_list = list(filter(lambda ticker_and_count: ticker_and_count[0] not in blacklist, top_list))
ticker_list = list(map(lambda t_a_c: t_a_c[0], top_list))

In [197]:
start = datetime(2014,1,1)
end = datetime(2021,1,1)

stock_data = {}
for ticker in ticker_list:
    stock = dr.data.DataReader(ticker, data_source='yahoo', start=start, end=end)
    stock_data[ticker] = stock

In [92]:
submission_sentiments = pd.read_csv("data/LabelledData/submissions_with_tickers_labelled.csv")
submission_sentiments.set_index("name", inplace=True)
comment_sentiments = pd.read_csv("data/LabelledData/comments_with_tickers_labelled.csv")

# Get Top
comment_sentiments["tickers"] = comment_sentiments["tickers"].apply(lambda x: literal_eval(str(x)))
comment_sentiments["ticker"] = comment_sentiments["tickers"].map(lambda tickers: tickers[0][0] if len(tickers) > 0 else np.NaN)
# print(ticker_list)
comment_sentiments = comment_sentiments[comment_sentiments["ticker"].isin(ticker_list)]
comment_sentiments.dropna()
# print(comment_sentiments["ticker"])

,Unnamed: 0,submission_name,id,text,score,created_utc,tickers,neg,neu,pos,ticker
1,1,t3_a4et2x,ebe85cf,Does anyone have a excel sample sheet they can...,3.0,1.544318e+09,"[[GOOG, 1]]",0.054193,0.913546,0.032261,GOOG
6,6,t3_an4n5g,efqslh9,Define cheap. For me I'm putting a bunch of mo...,3.0,1.549309e+09,"[[MSFT, 1]]",0.016944,0.317534,0.665522,MSFT
19,19,t3_bdmdvj,elfucsc,I am thinking about doing this strategy as wel...,2.0,1.555871e+09,"[[FB, 1]]",0.023970,0.929311,0.046719,FB
37,37,t3_cknban,evsth5n,MSFT TJX COST ADP APD HD AFL UNH VFC MKC,2.0,1.564756e+09,"[[MSFT, 1], [TJX, 1], [COST, 1], [ADP, 1], [AP...",0.019561,0.930602,0.049836,MSFT
55,55,t3_drbtts,f6h752b,MSFT and AAPL both pay dividends. You can set ...,8.0,1.572839e+09,"[[MSFT, 1], [AAPL, 1]]",0.017734,0.936916,0.045350,MSFT
...,...,...,...,...,...,...,...,...,...,...,...
41596,41596,t3_knnc52,ghm2c5k,Mgmt style difference.\n\nYou hear Adobe brag ...,3.0,1.609429e+09,"[[MSFT, 1]]",0.039295,0.924701,0.036005,MSFT
41597,41597,t3_knnc52,ghm73op,Azure will provide consistent growth for the n...,2.0,1.609431e+09,"[[MSFT, 2], [AMZN, 1]]",0.011626,0.846059,0.142315,MSFT
41598,41598,t3_knnc52,ghndxub,Ask yourself one question \n\nIs Microsoft off...,2.0,1.609454e+09,"[[MSFT, 1]]",0.014535,0.902674,0.082791,MSFT
41599,41599,t3_knnc52,ghlw67q,Tldr: MSFT to the moon 🚀🚀🚀🚀🚀🚀🚀,2.0,1.609425e+09,"[[MSFT, 1]]",0.018985,0.918247,0.062768,MSFT


In [212]:
avg_t_span = timedelta(weeks=1)
span_interval = timedelta(days=1)
future_t_interval = timedelta(days=3)

for filename in all_files:
    if not filename.endswith("submission.jsonl"):
        continue

    subreddit_path = filename.replace("_submission.jsonl", "")
    print(filename)
    
    df = pd.read_json(filename,lines=True)
    df["date"] = pd.to_datetime(df['created_utc'],unit='s')
    
    # clean dataset
    df["ticker"] = df["tickers"].map(lambda tickers: tickers[0][0] if len(tickers) > 0 else np.NaN)
    df = df[df["ticker"].isin(ticker_list)]
    df.dropna()
    df.set_index("date", inplace=True)
    df.sort_index(inplace=True)
    
    features = []
    
    t = start
    while t <= end - avg_t_span:
#         mask = (df >= t) & (df <= t + avg_t_span)
#         submissions = df.loc[mask]
#         print(df.index)
#         print(t)
        if(t.year != (t + avg_t_span).year):
            print(t)

        grouped_submissions = df[str(t):str(t+avg_t_span)]
        if len(grouped_submissions) == 0:
            t += span_interval
            continue
        
        comment_mask = comment_sentiments["submission_name"].isin(submissions["name"])
        comment_mask = comment_mask & comment_sentiments["ticker"].isin(ticker_list)
        grouped_comments = comment_sentiments.loc[comment_mask]
        
        for ticker in ticker_list:
            submissions = grouped_submissions[grouped_submissions["ticker"] == ticker]
            if len(submissions) == 0:
                continue
            
            comment_mask = grouped_comments["submission_name"].isin(submissions["name"])
            comment_mask = comment_mask & (grouped_comments["ticker"] == ticker)
            comments = grouped_comments.loc[comment_mask]
            
            s_scores = submissions["score"].values
            s_uv_ratio = submissions["upvote_ratio"].values

            s_sentiments = submission_sentiments.loc[submissions["name"],["neg","neu","pos"]].values
            s_sentiments *= (s_scores * s_uv_ratio).reshape(-1,1)
            s_sentiments = s_sentiments.sum(axis=0)

            if len(comments) == 0:
                c_sentiments = np.zeros(3)
            else:
                c_sentiments = comments[["neg","neu","pos"]].values
                c_scores = comments["score"].values
                c_sentiments = (c_sentiments * c_scores.reshape(-1,1)).sum(axis=0)
                
            # Get Stock Stuff
            stock = stock_data[ticker]
            cur_stock_idx = stock.index.get_loc(t, method="nearest")
            future_stock_idx = stock.index.get_loc(t + avg_t_span, method="nearest")
            cur_stock = stock.iloc[cur_stock_idx]["Open"] #Note : is rounded to nearest date
            future_stock = stock.iloc[future_stock_idx]["Close"]
            
            label = 1 if cur_stock <= future_stock else 0
            
            feature_vector = np.concatenate((s_sentiments, c_sentiments, np.array([label])), axis=0)
            features.append(feature_vector)
        
        t += span_interval
    pd.DataFrame(np.array(features)).to_csv(f"{subreddit_path}_features_labels.csv", index=False)
            


data/stocks_submission.jsonl
2014-12-25 00:00:00
2014-12-26 00:00:00
2014-12-27 00:00:00
2014-12-28 00:00:00
2014-12-29 00:00:00
2014-12-30 00:00:00
2014-12-31 00:00:00
2015-12-25 00:00:00
2015-12-26 00:00:00
2015-12-27 00:00:00
2015-12-28 00:00:00
2015-12-29 00:00:00
2015-12-30 00:00:00
2015-12-31 00:00:00
2016-12-25 00:00:00
2016-12-26 00:00:00
2016-12-27 00:00:00
2016-12-28 00:00:00
2016-12-29 00:00:00
2016-12-30 00:00:00
2016-12-31 00:00:00
2017-12-25 00:00:00
2017-12-26 00:00:00
2017-12-27 00:00:00
2017-12-28 00:00:00
2017-12-29 00:00:00
2017-12-30 00:00:00
2017-12-31 00:00:00
2018-12-25 00:00:00
2018-12-26 00:00:00
2018-12-27 00:00:00
2018-12-28 00:00:00
2018-12-29 00:00:00
2018-12-30 00:00:00
2018-12-31 00:00:00
2019-12-25 00:00:00
2019-12-26 00:00:00
2019-12-27 00:00:00
2019-12-28 00:00:00
2019-12-29 00:00:00
2019-12-30 00:00:00
2019-12-31 00:00:00
2020-12-25 00:00:00
data/stockmarket_submission.jsonl
2014-12-25 00:00:00
2014-12-26 00:00:00
2014-12-27 00:00:00
2014-12-28 00:00: